In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

In [2]:
model = load_model('./model.h5')

In [3]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

emotions = ["angry", "disgust", "fear", "happy", "sad", "surprise", "neutral"]


In [4]:
cap = cv2.VideoCapture(0)


In [5]:
while True:
    ret, frame = cap.read()

    if not ret:
        break
    frame = cv2.flip(frame, 1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=8, minSize=(30, 30))

    for (x, y, w, h) in faces:
        roi_gray = gray[y:y + h, x:x + w]
        roi_gray = cv2.resize(roi_gray, (48, 48), interpolation=cv2.INTER_AREA) 
        roi_gray = np.expand_dims(np.expand_dims(roi_gray, -1), 0)

        emotion_pred = model.predict(roi_gray)
        emotion_label = emotions[np.argmax(emotion_pred)]

        emoji_size = (200, 200)
        emoji_coords = (x + w // 2 - emoji_size[0] // 2 -200, y - emoji_size[1] // 2-20)

        emoji_path = f'./emoji/{emotion_label}.png'
        emoji = cv2.imread(emoji_path, cv2.IMREAD_UNCHANGED)

        if emoji is not None:
            emoji = cv2.resize(emoji, emoji_size, interpolation=cv2.INTER_AREA)

            alpha_s = emoji[:, :, 3] / 255.0
            alpha_l = 1.0 - alpha_s
            for c in range(0, 3):
                frame_region = frame[emoji_coords[1]:emoji_coords[1] + emoji_size[1], emoji_coords[0]:emoji_coords[0] + emoji_size[0], c]
                if emoji[:, :, c].shape == frame_region.shape:
                    frame[emoji_coords[1]:emoji_coords[1] + emoji_size[1], emoji_coords[0]:emoji_coords[0] + emoji_size[0], c] = (
                        alpha_s * emoji[:, :, c] + alpha_l * frame_region
                    )

    cv2.imshow('playing around', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


1/1 [==============================] - 0s 12ms/step


KeyboardInterrupt: 